In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

# define CIFAR10 Dataset and load
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
batch_size = 4
Train_Dataset = torchvision.datasets.CIFAR10(root = './data',train = True,
                                            download=True, transform=transform)

Test_Dataset = torchvision.datasets.CIFAR10(root = './data', train = False,
                                            download=True, transform=transform)

Train_loader = torch.utils.data.DataLoader(Train_Dataset, batch_size=batch_size,
                                           shuffle=True, num_workers=2)

Test_loader = torch.utils.data.DataLoader(Test_Dataset, batch_size = batch_size,
                                          shuffle=False, num_workers=2)

classes= ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')



  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [2]:
#Define Convolutional Neural Network
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3,6,5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6,16,5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)


  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x,1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x= self.fc3(x)
    return x

net = Net()


In [3]:
#Lost Function and Optimizer
import torch.optim as optim
learning_rate = 0.001
momentum = 0.9
weight = 0.01
criterion = nn.CrossEntropyLoss()

# without regularization
optimizer = optim.SGD(net.parameters(),lr=learning_rate,momentum=momentum)

In [4]:
#Training

# get some random training images
dataiter = iter(Train_loader)
images, labels = dataiter.next()

for epoch in range(2):
  loss = 0.0
  for i,data in enumerate(Train_loader,0):
    # get the inputs
    inputs,labels = data
    # zero the parameter gradients
    optimizer.zero_grad()

    outputs = net(inputs)
    loss_func = criterion(outputs,labels)
    loss_func.backward()
    optimizer.step()

    loss +=loss_func.item()
    if i % 1000 == 999:    # print every 1000 mini-batches
      print('[%d, %5d] loss: %.3f' %
      (epoch + 1, i + 1, loss / 2000))
      loss = 0.0


print('Training done !')

[1,  1000] loss: 1.151
[1,  2000] loss: 1.087
[1,  3000] loss: 0.983
[1,  4000] loss: 0.935
[1,  5000] loss: 0.853
[1,  6000] loss: 0.846
[1,  7000] loss: 0.798
[1,  8000] loss: 0.805
[1,  9000] loss: 0.779
[1, 10000] loss: 0.772
[1, 11000] loss: 0.748
[1, 12000] loss: 0.745
[2,  1000] loss: 0.708
[2,  2000] loss: 0.697
[2,  3000] loss: 0.697
[2,  4000] loss: 0.680
[2,  5000] loss: 0.683
[2,  6000] loss: 0.680
[2,  7000] loss: 0.677
[2,  8000] loss: 0.672
[2,  9000] loss: 0.649
[2, 10000] loss: 0.663
[2, 11000] loss: 0.655
[2, 12000] loss: 0.650
Training done !


In [5]:
#Accuracy
correct = 0
total = 0

with torch.no_grad():
  for data in Test_loader:
    images,labels = data
    outputs = net(images)

    _,predicted = torch.max(outputs.data,1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
accuracy = 100*(correct.item()/ total)

print('Accuracy of the network on the 10000 test images:',accuracy)

Accuracy of the network on the 10000 test images: 55.38999999999999


In [6]:
# prepare to count predictions for each class
correct_class = {classname: 0 for classname in classes}
total_class = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in Test_loader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)

        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_class[classes[label]] += 1
            total_class[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_class.items():
    accuracy_classes = 100 * float(correct_count) / total_class[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy_classes))

Accuracy for class plane is: 55.5 %
Accuracy for class car   is: 81.0 %
Accuracy for class bird  is: 26.4 %
Accuracy for class cat   is: 50.3 %
Accuracy for class deer  is: 54.8 %
Accuracy for class dog   is: 42.0 %
Accuracy for class frog  is: 62.0 %
Accuracy for class horse is: 64.1 %
Accuracy for class ship  is: 68.8 %
Accuracy for class truck is: 49.0 %
